<a href="https://colab.research.google.com/github/shiwkumar/UAV_CR/blob/main/CNN_DeepLearning_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc

class CNNSpectrumSensor:
    def __init__(self, num_secondary_users=4):
        self.num_sus = num_secondary_users
        self.sensing_time = 0.001  # 1ms sensing interval
        self.total_time = 60      # 1 minute simulation
        self.num_iterations = int(self.total_time / self.sensing_time)
        self.window_size = 128    # Input window size for CNN
        self.model = None

    def build_cnn_model(self):
        """Build CNN model for spectrum sensing"""
        model = models.Sequential([
            # Input layer
            layers.Input(shape=(self.window_size, 6)),  # [I, Q, SNR, path_loss, fading, position]

            # Convolutional layers
            layers.Conv1D(64, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling1D(2),

            layers.Conv1D(128, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling1D(2),

            layers.Conv1D(256, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling1D(2),

            # Dense layers
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

        return model

    def prepare_data(self, df):
        """Prepare data for CNN"""
        # Create feature matrix
        X = np.zeros((len(df), self.window_size, 6))

        for i in range(len(df)):
            # Generate I/Q samples based on SNR and channel conditions
            t = np.linspace(0, self.sensing_time, self.window_size)
            snr = df.iloc[i]['snr']
            path_loss = df.iloc[i]['path_loss']
            fading = df.iloc[i]['fading_magnitude']

            # Generate complex signal
            signal = np.exp(1j * 2 * np.pi * 1000 * t)  # 1 kHz signal
            signal = signal * np.sqrt(10**(snr/10))

            # Add channel effects
            signal = signal * fading * 10**(-path_loss/20)

            # Add noise
            noise = np.random.normal(0, 1, self.window_size) + 1j * np.random.normal(0, 1, self.window_size)
            noisy_signal = signal + noise

            # Fill feature matrix
            X[i, :, 0] = noisy_signal.real  # I component
            X[i, :, 1] = noisy_signal.imag  # Q component
            X[i, :, 2] = snr * np.ones(self.window_size)  # SNR
            X[i, :, 3] = path_loss * np.ones(self.window_size)  # Path loss
            X[i, :, 4] = fading * np.ones(self.window_size)  # Fading
            X[i, :, 5] = np.sqrt(df.iloc[i]['x']**2 + df.iloc[i]['y']**2 + df.iloc[i]['z']**2)  # Distance

        # Normalize features
        for i in range(X.shape[2]):
            scaler = StandardScaler()
            X[:, :, i] = scaler.fit_transform(X[:, :, i])

        y = df['primary_user_present'].values
        return X, y

def run_simulation():
    try:
        # Load dataset
        print("Loading dataset...")
        df = pd.read_hdf('cr_uav_dataset_10k.h5', key='data')

        # Initialize simulation parameters
        cnn_sensor = CNNSpectrumSensor(num_secondary_users=4)

        # Create SNR bins with wider ranges to ensure sufficient samples
        snr_bins = np.arange(-20, 6, 2)  # Changed step size to 2
        results = []

        # Print dataset statistics
        print(f"Total samples in dataset: {len(df)}")
        print(f"SNR range in dataset: [{df['snr'].min():.1f}, {df['snr'].max():.1f}]")

        # Run simulation for each SNR bin
        for snr_min, snr_max in zip(snr_bins[:-1], snr_bins[1:]):
            try:
                # Filter dataset for current SNR range
                df_snr = df[(df['snr'] >= snr_min) & (df['snr'] < snr_max)]

                print(f"Processing SNR range [{snr_min}, {snr_max}] dB with {len(df_snr)} samples")

                if len(df_snr) < 100:  # Reduced minimum sample requirement
                    print(f"Insufficient samples for SNR range [{snr_min}, {snr_max}] dB")
                    continue

                # Prepare data
                X, y = cnn_sensor.prepare_data(df_snr)

                # Split data
                split_idx = int(0.8 * len(X))
                X_train, X_test = X[:split_idx], X[split_idx:]
                y_train, y_test = y[:split_idx], y[split_idx:]

                # Build and train model
                cnn_sensor.model = cnn_sensor.build_cnn_model()

                # Train model
                history = cnn_sensor.model.fit(
                    X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=0
                )

                # Evaluate performance
                y_pred = cnn_sensor.model.predict(X_test, verbose=0)
                fpr, tpr, _ = roc_curve(y_test, y_pred)

                # Calculate cooperative detection (OR-rule for multiple SUs)
                coop_detection = 1 - np.prod([1-tpr[1] for _ in range(cnn_sensor.num_sus)])

                # Store results using average SNR of the bin
                avg_snr = (snr_min + snr_max) / 2
                results.append({
                    'snr': avg_snr,
                    'snr_min': snr_min,
                    'snr_max': snr_max,
                    'num_samples': len(df_snr),
                    'detection_prob': coop_detection,
                    'false_alarm_prob': fpr[1],
                    'training_accuracy': history.history['accuracy'][-1],
                    'validation_accuracy': history.history['val_accuracy'][-1]
                })

            except Exception as e:
                print(f"Error processing SNR range [{snr_min}, {snr_max}] dB: {str(e)}")
                continue

        if not results:
            raise ValueError("No valid results generated from any SNR range")

        # Convert results to DataFrame
        results_df = pd.DataFrame(results)

        # Sort results by SNR
        results_df = results_df.sort_values('snr')

        # Create plots
        plt.figure(figsize=(12, 8))
        plt.errorbar(results_df['snr'], results_df['detection_prob'],
                    xerr=(results_df['snr_max'] - results_df['snr_min'])/2,
                    fmt='o-', label='Cooperative Detection',
                    linewidth=2, markersize=8, capsize=5)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xlabel('SNR (dB)', fontsize=12)
        plt.ylabel('Probability of Detection', fontsize=12)
        plt.title(f'CNN-based Detection Performance vs SNR\n({cnn_sensor.num_sus} Secondary Users)',
                  fontsize=14, pad=20)
        plt.legend(fontsize=10)
        plt.xlim(min(results_df['snr'])-2, max(results_df['snr'])+2)
        plt.ylim(0, 1.1)
        plt.grid(True)
        plt.savefig('cnn_simulation_results.png', dpi=300, bbox_inches='tight')
        plt.close()

        # Plot training metrics
        plt.figure(figsize=(15, 5))

        # Training/Validation Accuracy
        plt.subplot(1, 3, 1)
        plt.plot(results_df['snr'], results_df['training_accuracy'], 'o-',
                label='Training', linewidth=2, markersize=6)
        plt.plot(results_df['snr'], results_df['validation_accuracy'], 's-',
                label='Validation', linewidth=2, markersize=6)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xlabel('SNR (dB)', fontsize=12)
        plt.ylabel('Accuracy', fontsize=12)
        plt.title('Model Accuracy vs SNR', fontsize=14)
        plt.legend(fontsize=10)
        plt.ylim(0, 1.1)

        # Sample Distribution
        plt.subplot(1, 3, 2)
        plt.bar(results_df['snr'], results_df['num_samples'])
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xlabel('SNR (dB)', fontsize=12)
        plt.ylabel('Number of Samples', fontsize=12)
        plt.title('Sample Distribution', fontsize=14)

        # ROC-like plot
        plt.subplot(1, 3, 3)
        plt.plot(results_df['false_alarm_prob'], results_df['detection_prob'], 'o-',
                linewidth=2, markersize=6, color='green')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xlabel('Probability of False Alarm', fontsize=12)
        plt.ylabel('Probability of Detection', fontsize=12)
        plt.title('Detection vs False Alarm', fontsize=14)
        plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
        plt.axis('square')
        plt.xlim(0, 1)
        plt.ylim(0, 1)

        plt.tight_layout()
        plt.savefig('cnn_training_metrics.png', dpi=300, bbox_inches='tight')
        plt.close()

        # Print performance summary
        print("\nCNN Simulation Results Summary")
        print("-----------------------------")
        print(f"Number of Secondary Users: {cnn_sensor.num_sus}")
        print(f"Total Simulation Time: {cnn_sensor.total_time} seconds")
        print(f"Sensing Interval: {cnn_sensor.sensing_time} seconds")
        print("\nPerformance Metrics:")
        print(results_df.round(4))

        # Save results
        results_df.to_csv('cnn_simulation_results.csv', index=False)

        return results_df

    except Exception as e:
        print(f"Simulation failed: {str(e)}")
        return None

if __name__ == "__main__":
    # Set random seeds for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)

    # Run simulation
    results_df = run_simulation()

Loading dataset...
Total samples in dataset: 10000
SNR range in dataset: [-20.0, 5.0]
Processing SNR range [-20, -18] dB with 801 samples
Processing SNR range [-18, -16] dB with 808 samples
Processing SNR range [-16, -14] dB with 745 samples


Processing SNR range [-14, -12] dB with 844 samples


Processing SNR range [-12, -10] dB with 800 samples
Processing SNR range [-10, -8] dB with 835 samples
Processing SNR range [-8, -6] dB with 779 samples
Processing SNR range [-6, -4] dB with 787 samples
Processing SNR range [-4, -2] dB with 750 samples
Processing SNR range [-2, 0] dB with 813 samples
Processing SNR range [0, 2] dB with 817 samples
Processing SNR range [2, 4] dB with 832 samples

CNN Simulation Results Summary
-----------------------------
Number of Secondary Users: 4
Total Simulation Time: 60 seconds
Sensing Interval: 0.001 seconds

Performance Metrics:
     snr  snr_min  snr_max  num_samples  detection_prob  false_alarm_prob  \
0  -19.0      -20      -18          801          0.0000            0.0133   
1  -17.0      -18      -16          808          0.0000            0.0122   
2  -15.0      -16      -14          745          0.0503            0.0000   
3  -13.0      -14      -12          844          0.0523            0.0000   
4  -11.0      -12      -10          80